In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Example: Unzip a file from Google Drive
# Replace '/content/drive/MyDrive/path/to/your_file.zip' with the actual path to your zip file
# Replace '/content/destination_folder' with the desired destination folder

!unzip '/content/drive/MyDrive/dio/projeto04/datasets.zip' -d '/content/drive/MyDrive/dio/projeto04/datasets'

A saída de streaming foi truncada nas últimas 5000 linhas.
  inflating: /content/drive/MyDrive/dio/projeto04/datasets/datasets/labels/hard_hat_workers1350.txt  
  inflating: /content/drive/MyDrive/dio/projeto04/datasets/datasets/labels/hard_hat_workers1351.txt  
  inflating: /content/drive/MyDrive/dio/projeto04/datasets/datasets/labels/hard_hat_workers1352.txt  
  inflating: /content/drive/MyDrive/dio/projeto04/datasets/datasets/labels/hard_hat_workers1353.txt  
  inflating: /content/drive/MyDrive/dio/projeto04/datasets/datasets/labels/hard_hat_workers1354.txt  
  inflating: /content/drive/MyDrive/dio/projeto04/datasets/datasets/labels/hard_hat_workers1355.txt  
  inflating: /content/drive/MyDrive/dio/projeto04/datasets/datasets/labels/hard_hat_workers1356.txt  
  inflating: /content/drive/MyDrive/dio/projeto04/datasets/datasets/labels/hard_hat_workers1357.txt  
  inflating: /content/drive/MyDrive/dio/projeto04/datasets/datasets/labels/hard_hat_workers1358.txt  
  inflating: /content/d

In [ ]:
from pathlib import Path
import os, shutil, random

# ↩️  coloque aqui a pasta que contém TODOS os .png e .txt
RAW_DIR  = Path("/content/drive/MyDrive/dio/projeto04/datasets/datasets")

# 📦  pasta de destino já formatada para o YOLO (será criada agora)
BASE_DIR = Path("/content/drive/MyDrive/dio/projeto04/test01")

IMGS_TRAIN = BASE_DIR / "images/train"
IMGS_VAL   = BASE_DIR / "images/val"
LBL_TRAIN  = BASE_DIR / "labels/train"
LBL_VAL    = BASE_DIR / "labels/val"

for p in [IMGS_TRAIN, IMGS_VAL, LBL_TRAIN, LBL_VAL]:
    p.mkdir(parents=True, exist_ok=True)
print("Estrutura de pastas criada em:", BASE_DIR)


Estrutura de pastas criada em: /content/drive/MyDrive/dio/projeto04/test01


 Separar 80 % treino / 20 % validação e copiar arquivos

In [ ]:
from pathlib import Path
import shutil, random

# Caminhos reais para imagens e rótulos
IMAGES_DIR = Path("/content/drive/MyDrive/dio/projeto04/datasets/datasets/images")
LABELS_DIR = Path("/content/drive/MyDrive/dio/projeto04/datasets/datasets/labels")

# Pasta base para saída formatada
BASE_DIR = Path("/content/drive/MyDrive/dio/projeto04/test01")
IMGS_TRAIN = BASE_DIR / "images/train"
IMGS_VAL   = BASE_DIR / "images/val"
LBL_TRAIN  = BASE_DIR / "labels/train"
LBL_VAL    = BASE_DIR / "labels/val"

# Criar estrutura de pastas
for p in [IMGS_TRAIN, IMGS_VAL, LBL_TRAIN, LBL_VAL]:
    p.mkdir(parents=True, exist_ok=True)

# Obter nomes dos arquivos (sem extensão)
image_stems = [f.stem for f in IMAGES_DIR.glob("*.png")]
random.seed(42)
random.shuffle(image_stems)

# Separar 80% treino / 20% validação
split_idx = int(len(image_stems) * 0.8)
train_stems = image_stems[:split_idx]
val_stems = image_stems[split_idx:]

def mover(lista, src_imgs, src_lbls, dst_imgs, dst_lbls):
    for stem in lista:
        img_src = src_imgs / f"{stem}.png"
        lbl_src = src_lbls / f"{stem}.txt"
        if img_src.exists() and lbl_src.exists():
            shutil.copy(img_src, dst_imgs / img_src.name)
            shutil.copy(lbl_src, dst_lbls / lbl_src.name)

mover(train_stems, IMAGES_DIR, LABELS_DIR, IMGS_TRAIN, LBL_TRAIN)
mover(val_stems, IMAGES_DIR, LABELS_DIR, IMGS_VAL, LBL_VAL)

print(f"Treino: {len(train_stems)} imagens")
print(f"Validação: {len(val_stems)} imagens")


Treino: 4682 imagens
Validação: 1171 imagens


In [ ]:
data_yaml_path = "/content/drive/MyDrive/dio/projeto04/test01/data.yaml"

yaml_content = """train: /content/drive/MyDrive/dio/projeto04/test01/images/train
val: /content/drive/MyDrive/dio/projeto04/test01/images/val

nc: 2
names: ['mask', 'helmet']
"""

with open(data_yaml_path, "w") as f:
    f.write(yaml_content)

print(f"'data.yaml' criado em: {data_yaml_path}")



'data.yaml' criado em: /content/drive/MyDrive/dio/projeto04/test01/data.yaml


In [2]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [3]:
!nvidia-smi


Sun Jul 27 19:14:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA disponível:", torch.cuda.is_available())
print("GPU em uso:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Nenhuma")


PyTorch version: 2.6.0+cu124
CUDA disponível: True
GPU em uso: Tesla T4


In [7]:
!yolo task=detect mode=train \
  model=yolov8n.pt \
  data=/content/drive/MyDrive/dio/projeto04/test01/data.yaml \
  epochs=10 imgsz=640 \
  project=/content/drive/MyDrive/dio/projeto04/resultados_yolo \
  name=treinamento_mascara_capacete


Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/dio/projeto04/test01/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=treinamento_mascara_capacete, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, pe

In [1]:
!ls

sample_data
